## Test Campaigns

Now that our campaigns have been fully created, let's test each campaign and evaluate the results.

## Setup

To get started, we need to perform a bit of setup.
Walk through each of the following steps to configure your environment to
interact with the Amazon Personalize Service.

In [ ]:
%store -r

In [ ]:
# Import Dependencies

import boto3
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import requests
import csv
import sys
import botocore
import uuid
from collections import defaultdict
import random
import numpy as np

from packaging import version
from botocore.exceptions import ClientError
from pathlib import Path

%matplotlib inline

# Setup Clients

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')

servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')


# The Uid is a unique ID and we need it to find the role made by CloudFormation
with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        Uid = tag['Value']
        break

print('Uid:', Uid)
print('Region:', ssm.meta.region_name)

### Test Related Product Recommendations Campaign

Let's test the recommendations made by the related items/products campaign by selecting a product from the Retail Demo Store's [Products](https://github.com/aws-samples/retail-demo-store/tree/master/src/products) microservice and requesting related item recommendations for that product.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

#### Select a Product

We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [ ]:
product_id = '020a5afe-fb13-4499-a1fa-8594d326eaa0'

response = requests.get('http://{}/products/id/{}'.format(products_service_instance, product_id))
product = response.json()
print(json.dumps(product, indent=4, sort_keys=True))

#### Get Related Product Recommendations for Product

Now let's call Amazon Personalize to get related item/product recommendations for our product from the related item campaign.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = related_campaign_arn,
    itemId = str(product_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']

In [ ]:
print(json.dumps(item_list, indent=4))

Since the `itemId`'s in the above response don't tell us much about the products being recommended, let's get detailed information for each item ID from the Products microservice.

In [ ]:
for item in item_list:
    response = requests.get('http://{}/products/id/{}'.format(products_service_instance, item['itemId']))
    print(json.dumps(response.json(), indent = 4))

Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the similar item recommendations from the SIMS recipe are based on the interactions we generated as input into the solution creation process above.

### Test Product Recommendations Campaign

Let's test the recommendations made by the product recommendations campaign by selecting a user from the Retail Demo Store's Users microservice and requesting item recommendations for that user.

#### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

In [ ]:
user_id = 256

response = requests.get('http://{}/users/id/{}'.format(users_service_instance, user_id))
user = response.json()
persona = user['persona']

In [ ]:
print(json.dumps(user, indent=4, sort_keys=True))

**Take note of the `persona` value for the user above. We should see recommendations for products consistent with this persona since we generated historical interactions for products in the categories represented in the persona.**

In [ ]:
print('Shopper persona for user {} is {}'.format(user_id, persona))

#### Get Product Recommendations for User

Now let's call Amazon Personalize to get recommendations for our user from the product recommendations campaign.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = recommend_campaign_arn,
    userId = str(user_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']

In [ ]:
print(json.dumps(item_list, indent=4))

Notice that in this response we have a `score` field returned with each `itemId`. For all recipes except SIMS and Popularity-Count, Personalize [calculates a score](https://docs.aws.amazon.com/personalize/latest/dg/getting-real-time-recommendations.html) for each recommended item. Score values are between 0.0 and 1.0 and the sum of all scores across all items in your interactions and items datasets will total to 1.0. Therefore, the absolute value of scores will be smaller for larger item catalogs. We'll see how scores are calculated a bit differently for the personalized-ranking recipe below.

Since the `itemId`'s in the above response don't tell us much about the products being recommended, let's retrieve product details from the Products microservice.

In [ ]:
print('User persona: ' + persona)

for item in item_list:
    response = requests.get('http://{}/products/id/{}'.format(products_service_instance, item['itemId']))
    print(json.dumps(response.json(), indent = 4))

Are the recommended products consistent with the persona? Note that this is a rather contrived example using a limited amount of generated interaction data without model parameter tuning. The purpose is to give you hands on experience building models and retrieving inferences from Amazon Personalize. 

### Test Personalized Ranking Campaign

Next let's evaluate the results of the personalized ranking campaign. As a reminder, given a list of items and a user, this campaign will rerank the items based on the preferences of the user. For the Retail Demo Store, we will use this campaign to rerank the products listed for each category and the featured products list as well as reranking catalog search results displayed in the search widget.

#### Get Featured Products List

First let's get the list of featured products from the Products microservice.

In [ ]:
response = requests.get('http://{}/products/featured'.format(products_service_instance))
featured_products = response.json()
print(json.dumps(featured_products, indent = 4))

#### ReRank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rerank for a specific user. This reranking will allow us to provide ranked products based on the user's behavior. These behaviors should be consistent the same persona that was mentioned above (since we're going to use the same `user_id`).

In [ ]:
unranked_product_ids = []

for product in featured_products:
    unranked_product_ids.append(product['id'])
    
print(', '.join(unranked_product_ids))

Now let's have Personalize rank the featured product IDs based on our random user.

In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=ranking_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

Are the reranked results different than the original results from the Search service? Notice that we are also given a score for each item but this time the score values are larger. This is because scores for personalized-ranking results are calculated just across the items being reranked. Experiment with a different `user_id` in the cells above to see how the item ranking changes.

#### Pick products for discount

Using the featured products list we'll pick some products for discount from the featured products.

We'll get the ranking when discount context is applied for comparison. This is a using the "contextual metadata" feature of Amazon Personalize.


In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=ranking_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user_id),
    context={'DISCOUNT': 'Yes'} # Here we provide the context for the ranking
)
disount_reranked = response['personalizedRanking']
print('Discount context ranking:', json.dumps(disount_reranked, indent = 4))
print('Discount:', [item['itemId'] for item in disount_reranked[:2]])

We could use the discount-context ranking directly, but what we might be more interested in seeing is those products that
benefit from having a discout shown. In our simulated data, certain products are more likely to see 
purchases with discount (to be precise, the cheaper ones). Let us find out which products benefit most. We also make use of the scores returned by Personalize when it returns the ranking.

In [ ]:
eps = 0.00001 #  "epsilon" - a number slightly more than zero so we don't get division by zero
non_discount_rerank_scores = {item['itemId']: max(item['score'], eps) for item in reranked}
discount_rerank_scores = {item['itemId']: item['score'] for item in disount_reranked}
score_increases_with_discount = {item_id: discount_rerank_scores[item_id]/non_discount_rerank_scores[item_id]
                                 for item_id in discount_rerank_scores}
# Let us get the sorted items:
discount_improve_sorted_items = sorted(score_increases_with_discount.keys(),
                                       key=lambda key: score_increases_with_discount[key])

print('Improvement ranking:', discount_improve_sorted_items)
# Let us pick the two items that respond best to discounts
print('Discount:', discount_improve_sorted_items[:2])

Has the ranking changed?

## Enable Campaigns in Retail Demo Store Recommendations Service

Now that we've tested our campaigns and can get related product, product recommendations, and reranked items for our users, we need to enable the campaigns in the Retail Demo Store's [Recommendations service](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations). The Recommendations service is called by the Retail Demo Store Web UI when a signed in user visits a page with personalized content capabilities (home page, product detail page, and category page). The Recommendations service checks Systems Manager Parameters values to determine the Personalize campaign ARNs to use for each of our three personalization use-cases.

Let's set the campaign ARNs for our campaigns in the expected parameter names.

### Update SSM Parameter To Enable Related Products

In [ ]:
response = ssm.put_parameter(
    Name='retaildemostore-related-products-campaign-arn',
    Description='Retail Demo Store Related Products Campaign Arn Parameter',
    Value='{}'.format(related_campaign_arn),
    Type='String',
    Overwrite=True
)

### Update SSM Parameter To Enable Product Recommendations

In [ ]:
response = ssm.put_parameter(
    Name='retaildemostore-product-recommendation-campaign-arn',
    Description='Retail Demo Store Product Recommendation Campaign Arn Parameter',
    Value='{}'.format(recommend_campaign_arn),
    Type='String',
    Overwrite=True
)

### Update SSM Parameter To Enable Search Personalization

In [ ]:
response = ssm.put_parameter(
    Name='retaildemostore-personalized-ranking-campaign-arn',
    Description='Retail Demo Store Personalized Ranking Campaign Arn Parameter',
    Value='{}'.format(ranking_campaign_arn),
    Type='String',
    Overwrite=True
)